# HuggingFace Local

- Author: [Min-su Jung](https://github.com/effort-type)
- Design: 
- Peer Review : [HyeonJong Moon](https://github.com/hj0302), [sunworl](https://github.com/sunworl)
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/04-Model/07-HuggingFaceLocal.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/04-Model/07-HuggingFaceLocal.ipynb)

## Overview  

This tutorial covers how to use **Hugging Face's open-source models** in a **local environment**, instead of relying on **paid API models** such as OpenAI, Claude, or Gemini.  

**Hugging Face Local Model** enables querying large language models (LLMs) using **computational resources from your local machine, such as CPU, GPU or TPU**, without relying on external cloud services.  

- **Advantages**  
    - No usage fees.  
    - Lower risk of data leakage.  

- **Disadvantages**  
    - Requires significant computational resources (e.g., **GPU/TPU**).  
    - Fine-tuning and inference require substantial time and resources.  

In this tutorial, we will create a simple example using `HuggingFacePipeline` to **run an LLM locally using the `model_id` of a publicly available model**.  

Note: Since this tutorial runs on a CPU, performance may be slower.

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Use Hugging Face Models](#Use-Hugging-Face-Models)
    - [Set Download Path for Hugging Face Models/Tokenizers](#Set-Download-Path-for-Hugging-Face-Models/Tokenizers)
    - [Hugging Face Model Configuration and Response Generation](#Hugging-Face-Model-Configuration-and-Response-Generation)
    - [Prompt Template and Chain Creation](#Prompt-Template-and-Chain-Creation)

### References

- [LangChain: HuggingFace Local Pipelines](https://python.langchain.com/docs/integrations/llms/huggingface_pipelines)
- [LangChain: PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#prompttemplate)
- [LangChain: LCEL](https://python.langchain.com/docs/concepts/lcel)
- [Hugging Face: Phi-3-mini-4k-instruct](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct)
----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
!pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_huggingface",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "HuggingFace-Local",
    }
)

Environment variables have been set successfully.


In [4]:
from dotenv import load_dotenv

load_dotenv(override=True)

False

## Use Hugging Face Models

### Set Download Path for Hugging Face Models/Tokenizers

Set the download path for Hugging Face models/tokenizers using the `os.environ["HF_HOME"]` environment variable.

- Configure it to download Hugging Face models/tokenizers to a desired local path, ex) `./cache/`

In [6]:
import os

os.environ["HF_HOME"] = "./cache/"

### Hugging Face Model Configuration and Response Generation

Assign the repo ID of the Hugging Face model to the `repo_id` variable.

- `microsoft/Phi-3-mini-4k-instruct` Model: https://huggingface.co/microsoft/Phi-3-mini-4k-instruct
- Use `invoke()` to generate a response using the Hugging Face model.

In [9]:
from langchain_huggingface import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 256,
        "top_k": 50,
        "temperature": 0.1,
        "do_sample": True,
    },
)

llm.invoke("Hugging Face is")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


'Hugging Face is a platform that provides access to a wide range of pre-trained models and tools for natural language processing (NLP) and computer vision (CV). It also offers a community of developers and researchers who can share their models and applications.\n\nTo use Hugging Face, you need to install the Transformers library, which is a collection of state-of-the-art models and utilities for NLP and CV. You can install it using pip:\n\n```\npip install transformers\n```\n\nThen, you can import the models you want to use from the transformers module. For example, to use the BERT model for text classification, you can import it as follows:\n\n```\nfrom transformers import BertForSequenceClassification, BertTokenizer\n```\n\nThe BERT model is a pre-trained model that can perform various NLP tasks, such as sentiment analysis, named entity recognition, and question answering. The model consists of two parts: the encoder and the classifier. The encoder is a stack of transformer layers t

### Prompt Template and Chain Creation

Create a chain using the LLM model with `LCEL (LangChain Expression Language)` syntax.

- Use `PromptTemplate.from_template()` to create a prompt instructing the model to summarize the given input text.
- Use LCEL syntax, such as `prompt | llm`, to build a chain where the LLM generates a response based on the created prompt.

In [7]:
%%time
from langchain_core.prompts import PromptTemplate

template = """Summarizes TEXT in simple bullet points ordered from most important to least important.
TEXT:
{text}

KeyPoints: """

# Create PromptTemplate
prompt = PromptTemplate.from_template(template)

# Create Chain
chain = prompt | llm

text = """A Large Language Model (LLM) like me, ChatGPT, is a type of artificial intelligence (AI) model designed to understand, generate, and interact with human language. These models are "large" because they're built from vast amounts of text data and have billions or even trillions of parameters. Parameters are the aspects of the model that are learned from training data; they are essentially the internal settings that determine how the model interprets and generates language. LLMs work by predicting the next word in a sequence given the words that precede it, which allows them to generate coherent and contextually relevant text based on a given prompt. This capability can be applied in a variety of ways, from answering questions and composing emails to writing essays and even creating computer code. The training process for these models involves exposing them to a diverse array of text sources, such as books, articles, and websites, allowing them to learn language patterns, grammar, facts about the world, and even styles of writing. However, it's important to note that while LLMs can provide information that seems knowledgeable, their responses are generated based on patterns in the data they were trained on and not from a sentient understanding or awareness. The development and deployment of LLMs raise important considerations regarding accuracy, bias, ethical use, and the potential impact on various aspects of society, including employment, privacy, and misinformation. Researchers and developers continue to work on ways to address these challenges while improving the models' capabilities and applications."""
print(f"input text:\n\n{text}")

input text:

A Large Language Model (LLM) like me, ChatGPT, is a type of artificial intelligence (AI) model designed to understand, generate, and interact with human language. These models are "large" because they're built from vast amounts of text data and have billions or even trillions of parameters. Parameters are the aspects of the model that are learned from training data; they are essentially the internal settings that determine how the model interprets and generates language. LLMs work by predicting the next word in a sequence given the words that precede it, which allows them to generate coherent and contextually relevant text based on a given prompt. This capability can be applied in a variety of ways, from answering questions and composing emails to writing essays and even creating computer code. The training process for these models involves exposing them to a diverse array of text sources, such as books, articles, and websites, allowing them to learn language patterns, gra

In [8]:
# Execute Chain
response = chain.invoke({"text": text})

# Print Results
print(response)

c:\Users\User\AppData\Local\pypoetry\Cache\virtualenvs\langchain-opentutorial-kGt3Gz_0-py3.11\Lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Summarizes TEXT in simple bullet points ordered from most important to least important.
TEXT:
A Large Language Model (LLM) like me, ChatGPT, is a type of artificial intelligence (AI) model designed to understand, generate, and interact with human language. These models are "large" because they're built from vast amounts of text data and have billions or even trillions of parameters. Parameters are the aspects of the model that are learned from training data; they are essentially the internal settings that determine how the model interprets and generates language. LLMs work by predicting the next word in a sequence given the words that precede it, which allows them to generate coherent and contextually relevant text based on a given prompt. This capability can be applied in a variety of ways, from answering questions and composing emails to writing essays and even creating computer code. The training process for these models involves exposing them to a diverse array of text sources, suc